In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import urllib.request
import matplotlib.pyplot as plt

In [2]:
list_metricas = ['average_clustering', 'coefficient_distribution_degree', 'density', 'average_degree', 'average_short_path']#, 'number_of_nodes']
#list_metricas = ['coefficient_distribution_degree']

list_variables = [
    'Valor_Ativo',
    'Patrimonio_Liquido',
    'Cotas_Emitidas',
    'Valor_Patrimonial_Cotas',
    'Percentual_Despesas_Taxa_Administracao',
    'Percentual_Despesas_Agente_Custodiante',
    'Percentual_Rentabilidade_Efetiva_Mes',
    'Percentual_Rentabilidade_Patrimonial_Mes',
    'Percentual_Dividend_Yield_Mes',
    'Percentual_Amortizacao_Cotas_Mes',
    'Total_Necessidades_Liquidez',
    'Disponibilidades',
    'Titulos_Publicos',
    'Titulos_Privados',
    'Fundos_Renda_Fixa',
    'Total_Investido',
    'Direitos_Bens_Imoveis',
    'Terrenos',
    'Imoveis_Renda_Acabados',
    'Imoveis_Renda_Construcao',
    'Imoveis_Venda_Acabados',
    'Imoveis_Venda_Construcao',
    'Outros_Direitos_Reais',
    'Acoes',
    'Debentures',
    'Bonus_Subscricao',
    'Certificados_Deposito_Valores_Mobiliarios',
    'Cedulas_Debentures',
    'Fundo_Acoes',
    'FIP',
    'FII',
    'FDIC',
    'Outras_Cotas_FI',
    'Notas_Promissorias',
    'Acoes_Sociedades_Atividades_FII',
    'Cotas_Sociedades_Atividades_FII',
    'CEPAC',
    'CRI',
    'Letras_Hipotecarias',
    'LCI',
    'LIG',
    'Outros_Valores_Mobliarios',
    'Valores_Receber',
    'Contas_Receber_Aluguel',
    'Contas_Receber_Venda_Imoveis',
    'Outros_Valores_Receber',
    'Rendimentos_Distribuir',
    'Taxa_Administracao_Pagar',
    'Taxa_Performance_Pagar',
    'Obrigacoes_Aquisicao_Imoveis',
    'Adiantamento_Venda_Imoveis',
    'Adiantamento_Alugueis',
    'Obrigacoes_Securitizacao_Recebiveis',
    'Instrumentos_Financeiros_Derivativos',
    'Provisoes_Contigencias',
    'Outros_Valores_Pagar',
    'Total_Passivo',
 ]

list_cat = [
    #'Mandato',
    #'Segmento_Atuacao',
    #'Publico_Alvo',
    #'Tipo_Gestao',
]

list_cat_2 = [
    'Mandato',
    'Segmento_Atuacao',
    'Publico_Alvo',
    'Tipo_Gestao',
]

ds_fii = xr.open_dataset('dataset/cvm/dataset_cvm_metrica_mensal.nc').fillna(0)
df_dados_fii = pd.read_csv('dataset/cvm/dataset_cvm_metrica_mensal_dados.csv', index_col='CNPJ_Fundo')

df_dados_fii['Periodo'] = (pd.to_datetime(df_dados_fii['Data_Referencia_Final']) - pd.to_datetime(df_dados_fii['Data_Referencia_Inicial'])) / np.timedelta64(1, 'M')

df_fii = ds_fii.sel(metrica=list_metricas).to_array().sel(variable=list_variables).stack(var=['variable', 'metrica']).to_pandas()
df_fii['TIR'] = df_dados_fii['TIR'].fillna(-1)
df_fii_cat =  df_dados_fii[list_cat_2]
df_fii = df_fii[(df_fii[('TIR','')] <= 1) & (df_fii[('TIR','')] >= -1)]

cnpj_meses = xr.open_dataset('dataset/cvm/dataset_cvm_metrica_mensal.nc').fillna(0).Acoes.sel(metrica='number_of_nodes').to_pandas()

In [3]:
def select_cols(numbers):
    list_cat = []
    grupos_fiis = df_dados_fii[list_cat].dropna().agg('-'.join, axis=1)

    set_cols = set()

    for grupo in np.unique(grupos_fiis.values):
        cnpj_grupo = grupos_fiis[grupos_fiis==grupo].index.values


        df_dados = df_fii[df_fii.index.isin(cnpj_grupo)]
        corr = df_dados.corr()
        cols = corr['TIR'][corr['TIR'].abs() >= 0.0].abs().sort_values(ascending=False).index.values[1:(numbers+1)]

        for t in cols:
            set_cols.add(t)

    return list(set_cols)

In [4]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import OneHotEncoder

num_variaveis = len(select_cols(5000))
num_meses = 75

clf_train = []
clf_train_cat = []

for i in range(1, num_variaveis):
    cols_corr = select_cols(5000)[:i]
    clf_train_i = []
    clf_train_cat_i = []
    
    j_new = 0
    for j in range(12, num_meses):
        print(f'var: {i} - meses: {j}')
        features = df_fii[df_fii.index.isin(cnpj_meses[cnpj_meses >= j].index.values)][ cols_corr ].values
        preprocessing = OneHotEncoder().fit_transform(df_fii_cat[df_fii_cat.index.isin(df_fii[df_fii.index.isin(cnpj_meses[cnpj_meses >= j].index.values)].index.values)]).toarray()
        features_cat = np.hstack((features, preprocessing))
        target = df_fii[df_fii.index.isin(cnpj_meses[cnpj_meses >= j].index.values)][('TIR','')]
        std_target = target.fillna(-1.).values.flatten()

        model = RandomForestRegressor(n_jobs=-1, oob_score=True, max_leaf_nodes=2, max_depth=4)

        clf_train_i.append(make_pipeline(StandardScaler(), model))
        clf_train_cat_i.append(make_pipeline(StandardScaler(), model))
        
        X_train, X_test, y_train, y_test = train_test_split(features, std_target, test_size=1/3, random_state=0)
        clf_train_i[j_new].fit(X_train, y_train)
        
        print(f'train: {mean_absolute_percentage_error(y_train, clf_train_i[j_new].predict(X_train))}, test: {mean_absolute_percentage_error(y_test, clf_train_i[j_new].predict(X_test))}')
        
        X_train, X_test, y_train, y_test = train_test_split(features_cat, std_target, test_size=1/3, random_state=0)
        clf_train_cat_i[j_new].fit(X_train, y_train)
        
        print(f'train: {mean_absolute_percentage_error(y_train, clf_train_cat_i[j_new].predict(X_train))}, test: {mean_absolute_percentage_error(y_test, clf_train_cat_i[j_new].predict(X_test))}')
    
        j_new += 1

    clf_train.append(clf_train_i)
    clf_train_cat.append(clf_train_cat_i)

var: 1 - meses: 12
train: 92164715455.80446, test: 1.115460561789489
train: 990031272232.862, test: 1.6401320902759384
var: 1 - meses: 13
train: 100529352759.50601, test: 194865677181.08627
train: 525318211165.83795, test: 1038863616725.4313
var: 1 - meses: 14
train: 84132870769.34373, test: 1.5617298235958854
train: 654058921039.4972, test: 1.7357256889445405
var: 1 - meses: 15
train: 1099515373.2042713, test: 1.6099970786494278
train: 914296746028.805, test: 2.0031690912710207
var: 1 - meses: 16
train: 413540715746.46936, test: 1.8175063356226686
train: 931352745197.5078, test: 1.9321184119680652
var: 1 - meses: 17
train: 150793582190.5872, test: 1.3787030843711006
train: 1066175826649.5712, test: 1.9383068628998241
var: 1 - meses: 18
train: 257519722083.6987, test: 1.4769468132848305
train: 530108627741.34375, test: 1.4764053923900724
var: 1 - meses: 19
train: 47290093955.97882, test: 91829625292.925
train: 183203993823.83115, test: 1624012281305.6782
var: 1 - meses: 20
train: 23682

train: 951981154357.9423, test: 1.858586118867441
var: 2 - meses: 17
train: 232487071505.4245, test: 1.4608941473373727
train: 1008901360899.965, test: 1.8925548538654273
var: 2 - meses: 18
train: 262478686478.51315, test: 1.4934573964661806
train: 367959204419.1613, test: 1.437416720004938
var: 2 - meses: 19
train: 46596300464.93881, test: 90482391845.42131
train: 101592864665.31819, test: 1549290853609.3462
var: 2 - meses: 20
train: 227366055281.97086, test: 440493151155.1546
train: 35553024429.34842, test: 908733391190.4106
var: 2 - meses: 21
train: 83641329136.49211, test: 1.0614242143890558
train: 1072163935662.7458, test: 1.484413322649756
var: 2 - meses: 22
train: 19816638149.101067, test: 1.0222431198706745
train: 1183887395311.8782, test: 1.797674529327434
var: 2 - meses: 23
train: 78842424464.60603, test: 1.1422101670286406
train: 934986233941.394, test: 1.3900418997066042
var: 2 - meses: 24
train: 5147846329.147121, test: 1.2502354423884057
train: 380802323002.4982, test: 1.

train: 9862488695.05341, test: 1.0338909533279954
train: 1167601980607.1636, test: 1.5120110564124785
var: 3 - meses: 22
train: 64148996274.81469, test: 1.07609726551697
train: 1255693884664.4424, test: 1.9225911576711967
var: 3 - meses: 23
train: 30688833728.738285, test: 1.1431444073067798
train: 891139416757.3998, test: 1.3142142394344132
var: 3 - meses: 24
train: 26614059536.404255, test: 1.2417198198152266
train: 401797635777.6231, test: 1.4306867065457565
var: 3 - meses: 25
train: 1.052642732572692, test: 1.0612424040289048
train: 1.320489957331774, test: 1.668017153324713
var: 3 - meses: 26
train: 1.380750648456882, test: 1.345316868053291
train: 1.720353970800022, test: 1.666532669631123
var: 3 - meses: 27
train: 1.101932201426681, test: 1.0976278541305102
train: 1.2937297910839727, test: 1.112822117050563
var: 3 - meses: 28
train: 1.2850275960015485, test: 1.021891165332987
train: 1.5561206176003286, test: 1.2641919652577693
var: 3 - meses: 29
train: 1.3190418690188768, test: 

train: 1.3904903304074798, test: 1.40876166480249
train: 1.6527508607520325, test: 1.597762387412874
var: 4 - meses: 27
train: 1.1650380867384187, test: 1.044765964607297
train: 1.2316809098021408, test: 1.1474598696019247
var: 4 - meses: 28
train: 1.6720362483660125, test: 1.264759442423635
train: 1.5243468290500275, test: 1.2614585569258896
var: 4 - meses: 29
train: 1.273930495374219, test: 1.1338545465367647
train: 1.346816018405343, test: 1.1473452513154507
var: 4 - meses: 30
train: 1.0489602099969473, test: 1.1469450219607376
train: 1.229709362551762, test: 1.214272744637425
var: 4 - meses: 31
train: 1.392496281411392, test: 1.1564968964312763
train: 1.266034053093656, test: 1.060378483291745
var: 4 - meses: 32
train: 0.9850625184869437, test: 1.0191647543120377
train: 1.1749514127046388, test: 1.0780809009422494
var: 4 - meses: 33
train: 1.11216881957603, test: 1.0873049903016025
train: 1.0992580835149457, test: 1.4650306467561067
var: 4 - meses: 34
train: 0.9816842150316017, tes

train: 1.8784246790108357, test: 1.242925184990305
train: 1.8804856596838024, test: 1.2455002535535273
var: 5 - meses: 32
train: 1.4991900301009886, test: 1.360986312252042
train: 1.3667830384894883, test: 1.2578279733418136
var: 5 - meses: 33
train: 1.1674460177425185, test: 1.2942987608040006
train: 1.1059777310082424, test: 1.2850120103726117
var: 5 - meses: 34
train: 1.1728203719660582, test: 1.5202617989771288
train: 1.5481647004184402, test: 1.5666007577913899
var: 5 - meses: 35
train: 1.4461233399977969, test: 1.2699302275049704
train: 1.9437066588740397, test: 1.7101596159883905
var: 5 - meses: 36
train: 1.6102818187331869, test: 1.6204704432141606
train: 1.957085742963548, test: 1.158400063304764
var: 5 - meses: 37
train: 1.7132897791294848, test: 1.749841627871223
train: 1.382175725626373, test: 1.6191762895837263
var: 5 - meses: 38
train: 1.52829985692497, test: 2.2428597549823412
train: 1.2446698314019655, test: 1.9956570946030239
var: 5 - meses: 39
train: 1.658160654287660

...

train: 1.2785929480879485, test: 1.0780725376002394
var: 274 - meses: 48
train: 1.2606728261939377, test: 0.9951989259429496
train: 1.2831946640428282, test: 0.9710713462782845
var: 274 - meses: 49
train: 1.3054232772509289, test: 1.4243370190258173
train: 1.1946104490767766, test: 1.4281131140482775
var: 274 - meses: 50
train: 1.1455803642173443, test: 1.2567838398798208
train: 1.0815676479419745, test: 1.1927235520919766
var: 274 - meses: 51
train: 1.3586031533929572, test: 1.1727197361365322
train: 1.2219956263750893, test: 1.172291488819654
var: 274 - meses: 52
train: 1.3633301764408412, test: 1.1616239323801723
train: 1.434736309787128, test: 1.198673344207001
var: 274 - meses: 53
train: 1.068189369461765, test: 1.148093269133833
train: 1.0675110783320603, test: 1.1329737582261783
var: 274 - meses: 54
train: 1.5168449426570336, test: 0.9831388115366246
train: 1.4287270931762643, test: 0.9020550646317697
var: 274 - meses: 55
train: 1.3100564405637058, test: 1.2184945178692228
train

train: 1.2441270277073482, test: 1.1253313731358947
train: 1.33908086614318, test: 1.223343976172036
var: 275 - meses: 52
train: 1.333045370295429, test: 1.3186644399534369
train: 1.3441225374963341, test: 1.2376148773023978
var: 275 - meses: 53
train: 1.1056857618091678, test: 1.0862392344300624
train: 1.1899401810591288, test: 1.0043705088689228
var: 275 - meses: 54
train: 1.533843537325061, test: 0.9077592830596621
train: 1.4946587142314405, test: 0.9593856816386107
var: 275 - meses: 55
train: 1.2202564050817875, test: 1.1954688086681824
train: 1.3170177824788663, test: 1.1832291056957207
var: 275 - meses: 56
train: 1.271122764161639, test: 1.1311052589195787
train: 1.3419142719109107, test: 1.1006190130989248
var: 275 - meses: 57
train: 1.3028387103734047, test: 1.0789306080378591
train: 1.449860258538677, test: 1.0897006933467648
var: 275 - meses: 58
train: 1.1614740314713563, test: 1.238536042824087
train: 1.1444018078746794, test: 1.556924390678547
var: 275 - meses: 59
train: 0.

train: 1.2566671144439738, test: 1.2928595677021122
train: 1.3615531185935767, test: 1.131530497082092
var: 276 - meses: 56
train: 1.2823293584076, test: 1.244561363012702
train: 1.3329175724772666, test: 1.125703237316864
var: 276 - meses: 57
train: 1.485181593433321, test: 1.106705640105014
train: 1.5295025040786991, test: 1.0816721852962987
var: 276 - meses: 58
train: 1.1729079139166199, test: 1.5438306148484728
train: 1.1860434112256784, test: 1.1967566929025013
var: 276 - meses: 59
train: 1.006122122868392, test: 1.5392874633693179
train: 0.9628425286716635, test: 1.4454338793138195
var: 276 - meses: 60
train: 1.2709641792338167, test: 1.074216443340879
train: 1.1992487938628689, test: 1.0103344395337968
var: 276 - meses: 61
train: 1.4279706077961734, test: 1.2695082629243288
train: 1.4231136125943293, test: 1.2161955247869034
var: 276 - meses: 62
train: 1.0100879672465801, test: 1.766246372956638
train: 0.9753460506096094, test: 0.9887451891817927
var: 276 - meses: 63
train: 1.02

train: 1.2004297462902422, test: 1.7362371934982823
var: 277 - meses: 59
train: 1.0070887014713747, test: 2.238405725416356
train: 1.0018229564711043, test: 1.6099342056759611
var: 277 - meses: 60
train: 1.3548622729693853, test: 0.9954159643365526
train: 1.3424173062143598, test: 1.0169608499352611
var: 277 - meses: 61
train: 1.4373107902067102, test: 1.2593884142587488
train: 1.3560297634100895, test: 1.3003622264003962
var: 277 - meses: 62
train: 0.9889077914715398, test: 1.2967167231660153
train: 0.9855647215621348, test: 1.5480257006549467
var: 277 - meses: 63
train: 1.0081452367538917, test: 1.225704966090184
train: 0.9872995713730233, test: 1.0861600838987215
var: 277 - meses: 64
train: 1.0087202238616444, test: 1.0962207628641563
train: 0.963606500515472, test: 1.1078449637999932
var: 277 - meses: 65
train: 0.9779758761379316, test: 1.187038426869976
train: 0.9966172029543712, test: 1.1280393349878908
var: 277 - meses: 66
train: 1.2519718753848736, test: 1.0563045772089596
trai

train: 0.9845364134259069, test: 1.04061152175364
train: 0.9578715927997665, test: 1.4969416879372108
var: 278 - meses: 63
train: 1.02275889140853, test: 1.155408650504463
train: 1.0321145516582741, test: 1.2397981142386472
var: 278 - meses: 64
train: 0.9768069003796302, test: 1.200754151554945
train: 0.9932835518322235, test: 1.0751091400042088
var: 278 - meses: 65
train: 0.9671596824602847, test: 1.1156156184310098
train: 0.9808261551015416, test: 1.1580753105481967
var: 278 - meses: 66
train: 1.1362558335658, test: 1.0888161457524494
train: 1.2054660580335332, test: 1.076230291716581
var: 278 - meses: 67
train: 1.0203798024568933, test: 1.0394929405071478
train: 1.0271481816245593, test: 1.086215176923893
var: 278 - meses: 68
train: 1.1428674759099502, test: 1.033829705731798
train: 1.177805451971211, test: 1.0568157105183211
var: 278 - meses: 69
train: 1.1754798350613123, test: 1.0708988270750577
train: 1.2010512964758469, test: 1.0764198680988777
var: 278 - meses: 70
train: 1.1079

In [22]:
for i in range(len(clf_train)):
    X_train, X_test, y_train, y_test = train_test_split(features[:,:i+1], std_target, test_size=1/3, random_state=0)
    
    for j in range(len(clf_train[i])):
        print(X_train.shape, y_train.shape, clf_train[i][j].n_features_in_)
        print(clf_train[i][j].score(X_train, y_train))

(154, 1) (154,) 1


ValueError: X has 1 features, but RandomForestRegressor is expecting 24 features as input.

In [10]:
clf_train_cat

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=4, max_leaf_nodes=2, n_jobs=-1,
                                       oob_score=True))])